In [4]:
import sys,os
import pandas as pd
import importlib
sys.path.append(os.path.abspath('..'))
from utils import preprocess_data


In [ ]:
#RELOAD :修改函数但是仍然用sys.modules 中缓存的，没有显示更新
# import importlib
# from utils import preprocess_data
# importlib.reload(preprocess_data)
# from utils.preprocess_data import split_change_stable

<module 'utils.preprocess_data' from 'd:\\Edu\\SelfPresentation_Multimodal_airbnb\\utils\\preprocess_data.py'>

# test on 24change (skipable):

In [33]:
dfparis2024_change=pd.read_csv("data_jo_processed\listings_jo_paris2024_change.csv")
print(len(dfparis2024_change))
display(dfparis2024_change.head())

FileNotFoundError: [Errno 2] No such file or directory: 'data_jo_processed\\listings_jo_paris2024_change.csv'

In [34]:
df=dfparis2024_change.copy()
df.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [8]:
df=dfparis2024_change.copy()
#"host_verifications",
var_host=["host_has_profile_pic","host_picture_url","host_identity_verified",
          "host_name","host_is_superhost", "number_of_reviews","review_scores_rating", "host_since",
          "host_about","host_response_time", "host_response_rate"]
print(len(var_host))
for var in var_host:
    print(df[var].notna().value_counts(),"\n")
    print(df[var].value_counts())
    print("-----------------------------------------------------------")


11
host_has_profile_pic
True    23757
Name: count, dtype: int64 

host_has_profile_pic
t    22138
f     1619
Name: count, dtype: int64
-----------------------------------------------------------
host_picture_url
True    23757
Name: count, dtype: int64 

host_picture_url
https://a0.muscache.com/defaults/user_pic-225x225.png?v=3                                                                                 1619
https://a0.muscache.com/im/pictures/user/User-50978178/original/f8b585d7-a644-46a3-bb64-7dc6ef49fde2.jpeg?aki_policy=profile_x_medium      307
https://a0.muscache.com/im/pictures/user/User-460047164/original/c35a1007-74d6-46fb-bcbd-6175433b7dc3.png?aki_policy=profile_x_medium      166
https://a0.muscache.com/im/pictures/user/User-564251645/original/e6b7d255-ff32-46ad-8ce2-0300c52c3171.jpeg?aki_policy=profile_x_medium     116
https://a0.muscache.com/im/pictures/user/f1221c8c-c1be-4947-aaa7-4fca4221b75c.jpg?aki_policy=profile_x_medium                              114
              

In [ ]:
# 检查：review_scores_rating
print(df.review_scores_rating.notna().value_counts())#60%无评分？
print(df.review_scores_rating.notna().value_counts()/len(df))#60%无评分？ 
print(df.review_scores_rating.describe())

review_scores_rating
False    13793
True      9191
Name: count, dtype: int64
review_scores_rating
False    0.600113
True     0.399887
Name: count, dtype: float64
count    9191.000000
mean        4.687557
std         0.511817
min         0.000000
25%         4.560000
50%         4.880000
75%         5.000000
max         5.000000
Name: review_scores_rating, dtype: float64


In [ ]:

# # host_since:增加一列换算成年
# df['host_since'] = pd.to_datetime(df['host_since'], errors='coerce')  # 转换为日期格式
# scrape_date='2024-06-12'
# today = pd.Timestamp(scrape_date)  #爬取时间
# # today = pd.Timestamp.today()# 自动使用今天日期

# # 计算距离天数
# df['days_since_host'] = (today - df['host_since']).dt.days

# # 将天数转为“年限（分段逻辑）”
# def convert_days_to_years(days):
#     if pd.isna(days):
#         return None
#     years = days / 365.25
#     if years < 0.5:
#         return 0
#     elif years < 1:
#         return 1
#     else:
#         return round(years)
# df['years_since_host'] = df['days_since_host'].apply(convert_days_to_years)


# # 缺失值处理与信号变量
# df['has_since'] = df['host_since'].notnull().astype(int)
# df['years_since_host'] = df['years_since_host'].fillna(0)



True

In [ ]:
# host_is_superhost :对缺失superhost的填f
df['host_is_superhost']=df['host_is_superhost'].fillna('f')

#review_scores_rating:对缺失评分（数值）增加一列has_rating：有为1，无为0;
# 双变量法：相当于把有/无评分的分开，所以原列中可以fillna(0); 区分信号存在/强度
df['has_rating'] = df['review_scores_rating'].notna().astype(int)
df['review_scores_rating'] = df['review_scores_rating'].fillna(0)


# host_since:增加一列years_since_host，少于半年填0，少于1年填1
def add_years_since_host(df, date_col='host_since', scrape_date='2024-06-12'):
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    today = pd.Timestamp(scrape_date) if scrape_date else pd.Timestamp.today()

    # 计算距今天数
    df['days_since_host'] = (today - df[date_col]).dt.days

    # 分段换算成年限
    def convert_days_to_years(days):
        if pd.isna(days):
            return None
        years = days / 365.25
        if years < 0.5:
            return 0
        elif years < 1:
            return 1
        else:
            return round(years)

    df['years_since_host'] = df['days_since_host'].apply(convert_days_to_years)

    # 缺失信号与填充
    if int(df.host_since.isna().sum())!=0:
        df['has_since'] = df[date_col].notnull().astype(int)
        df['years_since_host'] = df['years_since_host'].fillna(0)

    return df
df = add_years_since_host(df, date_col='host_since', scrape_date='2024-06-12')


# host_about：增加两列：has_host_about, len
df['has_host_about'] = df['host_about'].notna().astype(int)
# df['host_about'] = df['host_about'].fillna("nan")#无文本保留nan？
df['len']=df['host_about'].apply(lambda x : len(x.split()) if isinstance(x, str) else 0)



# host_response_time（类别） 缺失值填 no_response_time?
df["host_response_time"]=df["host_response_time"].fillna('no_response_time')

# host_response_rate（数值） 缺失值新增一列has_response_rate：有为1，无为0
df['has_response_rate'] = df['host_response_rate'].notna().astype(int)
df['host_response_rate'] = df['host_response_rate'].fillna(0)


In [ ]:
#bilan des var:
var_ok=["host_has_profile_pic","host_picture_url","host_identity_verified","host_name","number_of_reviews"]

#部分缺失或结构性缺失（missing not at random, MNAR）
var_toprocess=["host_is_superhost",
                "review_scores_rating","has_rating", 
                "host_since","years_since_host",
                "host_about","has_host_about", "len",
                "host_response_time",
                "host_response_rate","has_response_rate"]

In [ ]:
for var in var_toprocess:
    print(df[var].value_counts(dropna=False))
    print("-----------------------------------------------------------")


host_is_superhost
f    21500
t     2257
Name: count, dtype: int64
-----------------------------------------------------------
review_scores_rating
0.00    13974
5.00     4226
4.00      412
4.50      380
4.67      304
        ...  
3.96        1
3.76        1
3.48        1
2.33        1
1.50        1
Name: count, Length: 157, dtype: int64
-----------------------------------------------------------
has_rating
0    13973
1     9784
Name: count, dtype: int64
-----------------------------------------------------------
host_since
2015-12-10    309
2022-05-20    170
2023-12-18    137
2024-02-27    134
2023-05-02    134
             ... 
2011-05-21      1
2020-07-23      1
2022-01-24      1
2019-12-21      1
2022-10-30      1
Name: count, Length: 4342, dtype: int64
-----------------------------------------------------------
years_since_host
0     3448
9     3106
8     2613
10    2476
1     2084
2     1573
11    1539
7     1392
5     1308
6     1258
3      900
4      872
12     798
13     297
1

In [43]:
print(df.calculated_host_listings_count.isna().sum())
df.calculated_host_listings_count.describe()   
print(df.calculated_host_listings_count.dtype)       

0
int64


# paris2024:

In [2]:
dfparis2024_change=pd.read_csv("data_jo_2_grouped\listings_jo_paris2024_change.csv")
print(len(dfparis2024_change))


23757


C:\Users\yeliu\AppData\Local\Temp\ipykernel_3732\2210174176.py:1: DtypeWarning: Columns (77,79) have mixed types. Specify dtype option on import or set low_memory=False.
  dfparis2024_change=pd.read_csv("data_jo_2_grouped\listings_jo_paris2024_change.csv")


In [11]:
importlib.reload(preprocess_data)
from utils.preprocess_data import preprocess_host_variables
dfparis2024_change_processed=preprocess_host_variables(dfparis2024_change)
#处理语言偏慢，其他都非常快！



******************************HOST VARS******************************
PROCESS METHODS :
- host_is_sueprhost: fillna('f')
- review_scores_rating: 缺失严重，新增一列, 'has_rating:1/0'
- host_since: 新增1列years_since_host :float, 按照ab页面显示计算年数，0.5-1年填1， 0-0.5年填0 
- has_host_about:新增3列'has_host_about:1/0','lang:en/fr/other_langs','len:int',
- host_response_time: fillna('no_response_time') 
- host_response_rate:缺失严重，增加一列has_response_rate:1/0， fillna(0)
- calculated_host_listings_count : 新增1列'professional_host:1/0'
==========================VARS================================
['host_has_profile_pic', 'host_picture_url', 'host_identity_verified', 'host_name', 'number_of_reviews', 'host_is_superhost', 'review_scores_rating', 'has_rating', 'host_since', 'years_since_host', 'host_about', 'has_host_about', 'lang', 'len', 'host_response_time', 'host_response_rate', 'has_response_rate', 'calculated_host_listings_count', 'professional_host'] 


================= BALN PROCESSED VARIABLES =====================

In [8]:
dfparis2024_change_processed.to_csv("data_jo_3_processed/listings_jo_paris2024_change_processed.csv",index=False)


In [9]:
dfparis2024_change_processed.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

# london2024

In [12]:
importlib.reload(preprocess_data)
from utils.preprocess_data import preprocess_host_variables
dflondon2024_change=pd.read_csv("data_jo_2_grouped\listings_jo_london2024_change.csv")
dflondon2024_change_processed=preprocess_host_variables(dflondon2024_change)




******************************HOST VARS******************************
PROCESS METHODS :
- host_is_sueprhost: fillna('f')
- review_scores_rating: 缺失严重，新增一列, 'has_rating:1/0'
- host_since: 新增1列years_since_host :float, 按照ab页面显示计算年数，0.5-1年填1， 0-0.5年填0 
- has_host_about:新增3列'has_host_about:1/0','lang:en/fr/other_langs','len:int',
- host_response_time: fillna('no_response_time') 
- host_response_rate:缺失严重，增加一列has_response_rate:1/0， fillna(0)
- calculated_host_listings_count : 新增1列'professional_host:1/0'

==========================VARS================================
['host_has_profile_pic', 'host_picture_url', 'host_identity_verified', 'host_name', 'number_of_reviews', 'host_is_superhost', 'review_scores_rating', 'has_rating', 'host_since', 'years_since_host', 'host_about', 'has_host_about', 'lang', 'len', 'host_response_time', 'host_response_rate', 'has_response_rate', 'calculated_host_listings_count', 'professional_host'] 


================= BALN PROCESSED VARIABLES ====================

# 完整数据处理+保存:

In [ ]:
import pandas as pd
import sys, os
import importlib
sys.path.append(os.path.abspath(".."))

from utils import preprocess_data
importlib.reload(preprocess_data)
from utils.preprocess_data import preprocess_host_variables



dfparis2024=pd.read_csv("data_jo_2_grouped\listings_jo_paris2024.csv")
dfparis2024_processed=preprocess_host_variables(dfparis2024)


C:\Users\yeliu\AppData\Local\Temp\ipykernel_3732\3519531814.py:4: DtypeWarning: Columns (77,79) have mixed types. Specify dtype option on import or set low_memory=False.
  dfparis2024=pd.read_csv("data_jo_2_grouped\listings_jo_paris2024.csv")




******************************HOST VARS******************************
PROCESS METHODS :
- host_is_sueprhost: fillna('f')
- review_scores_rating: 缺失严重，新增一列, 'has_rating:1/0'
- host_since: 新增1列years_since_host :float, 按照ab页面显示计算年数，0.5-1年填1， 0-0.5年填0 
- has_host_about:新增3列'has_host_about:1/0','lang:en/fr/other_langs','len:int',
- host_response_time: fillna('no_response_time') 
- host_response_rate:缺失严重，增加一列has_response_rate:1/0， fillna(0)
- calculated_host_listings_count : 新增1列'professional_host:1/0'

==========================VARS================================
['host_has_profile_pic', 'host_picture_url', 'host_identity_verified', 'host_name', 'number_of_reviews', 'host_is_superhost', 'review_scores_rating', 'has_rating', 'host_since', 'years_since_host', 'host_about', 'has_host_about', 'lang', 'len', 'host_response_time', 'host_response_rate', 'has_response_rate', 'calculated_host_listings_count', 'professional_host'] 


================= BALN PROCESSED VARIABLES ====================

In [14]:
dfparis2024_processed.to_csv("data_jo_3_processed\listings_jo_paris2024_processed.csv",index=False)

In [15]:
dflondon2024=pd.read_csv("data_jo_2_grouped\listings_jo_london2024.csv")
dflondon2024_processed=preprocess_host_variables(dflondon2024)
dflondon2024_processed.to_csv("data_jo_3_processed\listings_jo_london2024_processed.csv",index=False)



******************************HOST VARS******************************
PROCESS METHODS :
- host_is_sueprhost: fillna('f')
- review_scores_rating: 缺失严重，新增一列, 'has_rating:1/0'
- host_since: 新增1列years_since_host :float, 按照ab页面显示计算年数，0.5-1年填1， 0-0.5年填0 
- has_host_about:新增3列'has_host_about:1/0','lang:en/fr/other_langs','len:int',
- host_response_time: fillna('no_response_time') 
- host_response_rate:缺失严重，增加一列has_response_rate:1/0， fillna(0)
- calculated_host_listings_count : 新增1列'professional_host:1/0'

==========================VARS================================
['host_has_profile_pic', 'host_picture_url', 'host_identity_verified', 'host_name', 'number_of_reviews', 'host_is_superhost', 'review_scores_rating', 'has_rating', 'host_since', 'years_since_host', 'host_about', 'has_host_about', 'lang', 'len', 'host_response_time', 'host_response_rate', 'has_response_rate', 'calculated_host_listings_count', 'professional_host'] 


================= BALN PROCESSED VARIABLES ====================

In [31]:
# 检查 host_picture_url 是否有效，新增一列pic_url_is_valid
import requests
from requests.exceptions import RequestException
import time
from tqdm import tqdm
tqdm.pandas()  # 这行让 pandas 的 apply() 支持进度条



def check_pic_url(url, timeout=3):
    """
    检查头像 URL 是否有效（返回状态码200或类似）。
    返回 True 表示有效，False 表示无效或出错。
    """
    if not isinstance(url, str) or url.strip() == '':
        return False
    try:
        response = requests.head(url, timeout=timeout, allow_redirects=True)
        return response.status_code == 200 # 200则为True
    except RequestException:
        return False

start_time=time.time()
print(f'Check {len(df)} host pic url...')
# sample = df.sample(10, random_state=42).copy()
# sample['pic_url_is_valid'] = sample['host_picture_url'].progress_apply(check_pic_url)
df['host_pic_valid']=df['host_picture_url'].progress_apply(check_pic_url)
print(df['pic_url_is_valid'].value_counts(),'\n')

end_time=time.time()
print(f"check {len(df)} host pic : {end_time-start_time:.2f}s.")



Check 22984 host pic url...


  1%|          | 136/22984 [02:52<8:03:40,  1.27s/it] 


KeyboardInterrupt: 